LLM application through query routing and long-context, a type of reflection flow based on the user query
===

RAG routing is based on the complexity of the query and the context. The routing follows these rules:

1. Simple query, direct answer, no routing.
2. Answerable question, but complex query, then routing for query decomposition.
3. Route to long-context query for unanswerable questions.

Origin paper: [Retrieval Augmented Generation or Long-Context LLMs? A Comprehensive Study and Hybrid Approach](https://arxiv.org/abs/2407.16833)

Implementation: LLama-Index


In [1]:
from rich.pretty import pprint as pp
from icecream import ic
from IPython.display import Markdown

import nest_asyncio
nest_asyncio.apply()

## Const

In [2]:
from llama_index.llms.groq import Groq
from llama_index.core.response_synthesizers import ResponseMode

llm = "llama-3.1-70b-versatile"
embeds = "models/text-embedding-004"
chunk_size = 512
MAX_TOKENS = 2048
data_urls = [
    "https://hermesworld.com/de/karriere/jobs/Junior-Manager-mwd-HR-Controlling-de-j4364.html",
    "https://hermesworld.com/de/karriere/jobs/Senior-Product-Manager-mwd-de-j3839.html",
]
rerank_top_k = 10
similarity_top_k = 10
num_multi_steps = 4
verbose = True
streaming = False
response_mode = ResponseMode.TREE_SUMMARIZE
# https://github.com/run-llama/llama_index/blob/71c2cfdfbec6bcdd71f0e39f5dbb52c0e9f68ae5/llama-index-core/llama_index/core/response_synthesizers/type.py

## Token counting

In [3]:
from llama_index.core.callbacks import TokenCountingHandler
import tiktoken

token_counter = TokenCountingHandler(
    tokenizer=tiktoken.encoding_for_model("gpt-3.5-turbo").encode
)

In [4]:
def print_token_counter(counter: TokenCountingHandler):
    pp(
        (
            "Embedding Tokens: ",
            counter.total_embedding_token_count,
            "LLM Prompt Tokens: ",
            counter.prompt_llm_token_count,
            "LLM Completion Tokens: ",
            counter.completion_llm_token_count,
            "Total LLM Token Count: ",
            counter.total_llm_token_count,
        )
    )

    counter.reset_counts()


print_token_counter(token_counter)

('Embedding Tokens: ', 0, 'LLM Prompt Tokens: ', 0, 'LLM Completion Tokens: ', 0, 'Total LLM Token Count: ', 0)

## LLama-Index setting

In [5]:
from llama_index.core import Settings
from llama_index.embeddings.gemini import GeminiEmbedding
from llama_index.llms.gemini import Gemini
from llama_index.core.callbacks import CallbackManager

Settings.llm = Groq(model=llm, temperature=0, max_tokens=MAX_TOKENS)
Settings.embed_model = GeminiEmbedding(model_name=embeds)
Settings.callbacks = CallbackManager([token_counter])

In [6]:
from llama_index.readers.web import SimpleWebPageReader
import tempfile
from llama_parse import LlamaParse

origin_documents = SimpleWebPageReader(html_to_text=True).load_data(data_urls)
# pp(origin_documents)
document_contents = "\n".join(
    [f"----document 1:----\n\n{doc.text}" for doc in origin_documents]
)
# print(document_contents)
temp_file = tempfile.NamedTemporaryFile(delete=True, suffix=".txt")
temp_file.write(document_contents.encode("utf-8"))
temp_file.seek(0) 
ic(temp_file.name)
print(temp_file.read()) 
parser = LlamaParse(
    # api_key= api_key,
    result_type="markdown"
) 
documents = parser.load_data(temp_file.name)
pp(documents)
temp_file.close()

ic| temp_file.name: '/tmp/tmpf_5ek4lj.txt'


b'----document 1:----\n\n[ Privatkunden ](https://www.myhermes.de)\n\n\xc3\x96sterreich China Deutschland Vereinigtes K\xc3\xb6nigreich International USA\n\nMenu AT CN DE EN INT US [Hermes](https://www.hermesworld.com/de/)\n\n  * navigation.search\n\n  * [Start](https://www.hermesworld.com/de/)\n  * [Unsere Dienstleistungen](https://www.hermesworld.com/de/unsere-dienstleistungen/)\n\n    * [Transport Logistics](https://www.hermesworld.com/de/unsere-dienstleistungen/transport-logistics/)\n      * [Supply Chain Solutions](https://www.hermesworld.com/de/unsere-dienstleistungen/transport-logistics/supply-chain-solutions/unsere-fokusbranchen/)\n        * [Unsere Fokusbranchen](https://www.hermesworld.com/de/unsere-dienstleistungen/transport-logistics/supply-chain-solutions/unsere-fokusbranchen/)\n        * [Supply Chain Services](https://www.hermesworld.com/de/unsere-dienstleistungen/transport-logistics/supply-chain-solutions/supply-chain-services/)\n        * [E-Services](https://www.herme

[
│   Document(
│   │   id_='205d1cc0-ad0c-4bea-b424-cc30a626bbea',
│   │   embedding=None,
│   │   metadata={},
│   │   excluded_embed_metadata_keys=[],
│   │   excluded_llm_metadata_keys=[],
│   │   relationships={},
│   │   text='# Privatkunden\n\nÖsterreich China Deutschland Vereinigtes Königreich International USA\n\nMenu AT CN DE EN INT US Hermes\n\n- navigation.search\n- Start\n- Unsere Dienstleistungen',
│   │   mimetype='text/plain',
│   │   start_char_idx=None,
│   │   end_char_idx=None,
│   │   text_template='{metadata_str}\n\n{content}',
│   │   metadata_template='{key}: {value}',
│   │   metadata_seperator='\n'
│   ),
│   Document(
│   │   id_='38cd6b87-b9cb-4a0a-9b29-574e89846ce9',
│   │   embedding=None,
│   │   metadata={},
│   │   excluded_embed_metadata_keys=[],
│   │   excluded_llm_metadata_keys=[],
│   │   relationships={},
│   │   text='# Zusammenarbeit\n\nhttps://www.hermesworld.com/de/unsere-dienstleistungen/distribution/2-mann-handling/zusammenarbeit/\n\n# Dropshipping\n\nhttps://www.hermesworld.com/de/unsere-dienstleistungen/distribution/2-mann-handling/dropshipping/\n\n# Unser Service für Versender\n\nhttps://www.hermesworld.com/de/unsere-dienstleistungen/distribution/2-mann-handling/unser-service-fuer-versender/\n\n# Unser Service für Endkunden\n\nhttps://www.hermesworld.com/de/unsere-dienstleistungen/distribution/2-mann-handling/unser-service-fuer-endkunden/\n\n# CO2-neutraler Versand\n\nhttps://www.hermesworld.com/de/unsere-dienstleistungen/distribution/2-mann-handling/klimaneutraler-versand/\n\n# HES Pay\n\nhttps://www.hermesworld.com/de/unsere-dienstleistungen/distribution/2-mann-handling/hes-pay/\n\n# Distribution in Europa\n\nhttps://www.hermesworld.com/de/unsere-dienstleistungen/distribution/distribution-in-europa/\n\n# Über uns\n\n# Auf einen Blick\n\nhttps://www.hermesworld.com/de/ueber-uns/auf-einen-blick/hermes-auf-einen-blick/\n\n# Hermes Gruppe\n\n# Hermes Einrichtungs Service\n\nhttps://www.hermesworld.com/de/ueber-uns/hermes-gruppe/hermes-einrichtungs-service/\n\n- Hermes Einrichtungs Service\n- Logistikzentren\n- Auslieferdepots\n\n# Hermes Fulfilment\n\nhttps://www.hermesworld.com/de/ueber-uns/hermes-gruppe/hermes-fulfilment/hermes-fulfilment/\n\n- Hermes Fulfilment\n- Daten & Fakten\n- Einblicke\n- Nachhaltigkeit\n\n# Hermes Logistik Österreich\n\nhttps://www.hermesworld.com/de/ueber-uns/hermes-gruppe/hermes-logistik-at/hermes-logistik-oesterreich/\n\n- Hermes Logistik Österreich\n- PaketShop Partner\n\n# Hermes Germany GmbH\n\nhttps://www.hermesworld.com/de/ueber-uns/hermes-gruppe/hermes-logistik-gruppe-deutschland/hermes-logistik-gruppe-deutschland/\n\n- Hermes Germany GmbH\n- PaketShop-Partner werden\n- Vertragspartner werden\n\n- Formular\n\n# Hermes NexTec\n\nhttps://www.hermesworld.com/de/ueber-uns/hermes-gruppe/hermes-nextec/\n\n# Hermes Germany\n\nhttps://www.hermesworld.com/de/ueber-uns/hermes-',
│   │   mimetype='text/plain',
│   │   start_char_idx=None,
│   │   end_char_idx=None,
│   │   text_template='{metadata_str}\n\n{content}',
│   │   metadata_template='{key}: {value}',
│   │   metadata_seperator='\n'
│   ),
│   Document(
│   │   id_='62690241-1763-4ed6-a068-5404a6220a58',
│   │   embedding=None,
│   │   metadata={},
│   │   excluded_embed_metadata_keys=[],
│   │   excluded_llm_metadata_keys=[],
│   │   relationships={},
│   │   text='# gruppe/hermes-transport-logistics/hermes-transport-logistics/\n\n- Hermes Germany\n- Downloads\n\n# Hermes UK\n\n- Hermes UK\n- Kundenstimmen\n\n# Hermes BorderGuru\n\n- Hermes BorderGuru\n\n# Management\n\n- Management\n\n# Klima- und Umweltschutz\n\n- Klima- und Umweltschutz\n- Grüne Zustellung\n- Energieeffiziente Standorte\n- Unterstützung von Klimaschutzprojekten\n\n# Soziale Verantwortung\n\n- Faire Zusammenarbeit\n- Einhaltung von Sozialstandards\n- Gesellschaftliches Engagement\n\n# Downloads\n\n- Plattformen & Bezahlfunktionen China\n- Export nach China\n\n# Messen\n\n- Messen\n\n# Historie\n\n- 2010er Jahre\n- 2000er Jahre\n

## Setup step methods

### Chain

In [26]:
from llama_index.core import get_response_synthesizer

synthesizer = get_response_synthesizer(
    response_mode=response_mode, streaming=streaming
)

### Prompting

Re-prompt the default prompt for German input data. This can enhance query performance.

In [27]:
from langchain import hub
from llama_index.core import PromptTemplate

prompt_str = hub.pull("hwchase17/llama-rag").template.replace("context", "context_str").replace("question", "query_str")
prompt_str = prompt_str.replace("[INST] <<SYS>>", "[INST] <<SYS>> You contains data in German, thinking everything including query and ansewr in German if possible. ")
pp(prompt_str)

prompt_tmpl = PromptTemplate(prompt_str)

'[INST] <<SYS>> You contains data in German, thinking everything including query and ansewr in German if possible. Answer the users query_str only taking into account the following context_str. If the user asks for information not found in the below context_str, do not answer.\n\n<context_str>\n{context_str}\n</context_str>\n<</SYS>>\n\n {query_str} [/INST]'

### Top K RAG

In [28]:
from llama_index.core import VectorStoreIndex
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.postprocessor import SentenceTransformerRerank


splitter = SentenceSplitter(chunk_size=chunk_size)
vector_index = VectorStoreIndex.from_documents(documents, transformations=[splitter])
vector_query_engine = vector_index.as_query_engine(
    similarity_top_k=similarity_top_k,
    response_synthesizer=synthesizer,
    node_postprocessors=[
        SentenceTransformerRerank(top_n=rerank_top_k, model="BAAI/bge-reranker-base")
    ],
    summary_template=prompt_tmpl,
)
vector_query_engine.update_prompts(
    {"response_synthesizer:summary_template": prompt_tmpl}
)

In [29]:
prompts_dict = vector_query_engine.get_prompts()
pp(list(prompts_dict.values()))

[
│   PromptTemplate(
│   │   metadata={'prompt_type': <PromptType.CUSTOM: 'custom'>},
│   │   template_vars=['context_str', 'query_str'],
│   │   kwargs={},
│   │   output_parser=None,
│   │   template_var_mappings=None,
│   │   function_mappings=None,
│   │   template='[INST] <<SYS>> You contains data in German, thinking everything including query and ansewr in German if possible. Answer the users query_str only taking into account the following context_str. If the user asks for information not found in the below context_str, do not answer.\n\n<context_str>\n{context_str}\n</context_str>\n<</SYS>>\n\n {query_str} [/INST]'
│   )
]

### Multi-step query

In [30]:
from llama_index.core.indices.query.query_transform.base import (
    StepDecomposeQueryTransform,
)
from llama_index.core.query_engine import MultiStepQueryEngine

step_decompose_transform = StepDecomposeQueryTransform(verbose=verbose)
multi_steps_query_engine = MultiStepQueryEngine(
    query_engine=vector_query_engine,
    query_transform=step_decompose_transform,
    response_synthesizer=synthesizer,
    num_steps=num_multi_steps,
)

### Long context query

In [31]:
from llama_index.core.llms.llm import LLM
from llama_index.core.query_engine import CustomQueryEngine
from typing import Any
from langchain import hub
from llama_index.core import PromptTemplate


class VanillaQueryEngine(CustomQueryEngine):
    """RAG String Query Engine."""

    llm: LLM
    context: str
    prompt_tmpl: PromptTemplate

    def __call__(self, *args: Any, **kwds: Any) -> str:
        return self.custom_query(*args, **kwds)

    def custom_query(self, query_str: str) -> str:
        full_query = self.prompt_tmpl.format(
            context_str=self.context, query_str=query_str
        )
        return str(self.llm.complete(full_query))


lc_query_engine = VanillaQueryEngine(
    response_synthesizer=synthesizer,
    llm=Settings.llm,
    context="\n".join([doc.text for doc in documents]),
    prompt_tmpl=prompt_tmpl,
)

## Query with methods

### Simple query for try

In [73]:
query = "Tell me the jobs you know at Hermes Group?"

### Top K RAG

In [33]:
vc_res = vector_query_engine.query(query)
print(vc_res.response)

Ich habe Daten über die Hermes Gruppe, einem international tätigen Logistikunternehmen. Die Daten umfassen Informationen über die Unternehmenstätigkeit, Dienstleistungen, Karriere, Nachhaltigkeit, Klima- und Umweltschutz, sowie über die verschiedenen Gesellschaften innerhalb der Hermes Gruppe, wie Hermes Germany, Hermes UK, Hermes Logistik Österreich und Hermes Fulfilment. Ich habe auch Daten über die verschiedenen Dienstleistungen, wie Sendungsverfolgung, PaketShop finden, myHermes Business-Portal und click2supplychain.com.


### Multi-step query

In [34]:
ms_res = multi_steps_query_engine.query(query)
print(ms_res.response)

> Current query: What data do you have?
> New query: Since there is no context information provided, we cannot extract any relevant information to answer the original question or a subcomponent of it. Therefore, the next question would be the same as the original question, which is:

What data do you have?
> Current query: What data do you have?
> New query: What information do you have about the Hermes Gruppe?
> Current query: What data do you have?
> New query: What information do you have about the Hermes Germany GmbH?
> Current query: What data do you have?
> New query: What information do you have about Hermes Fulfilment?
Ich habe Daten über die Hermes Gruppe, einem international tätigen Logistikunternehmen. Die Daten umfassen Informationen über die Unternehmenstätigkeit, Dienstleistungen, Karrieremöglichkeiten, Nachhaltigkeit und Umweltschutz. Es gibt auch spezifische Informationen über die verschiedenen Gesellschaften innerhalb der Hermes Gruppe, wie Hermes Germany, Hermes Fulfi

In [35]:
def show_multi_steps(ms_res):
    sub_qa = ms_res.metadata["sub_qa"]
    tuples = [(t[0], t[1].response) for t in sub_qa]
    pp(tuples)

show_multi_steps(ms_res)

[
│   (
│   │   'Since there is no context information provided, we cannot extract any relevant information to answer the original question or a subcomponent of it. Therefore, the next question would be the same as the original question, which is:\n\nWhat data do you have?',
│   │   'Ich habe Daten über die Hermes Gruppe, einem international tätigen Logistikunternehmen. Die Daten umfassen Informationen über die Unternehmenstätigkeit, Dienstleistungen, Karrieremöglichkeiten, Nachhaltigkeit und Umweltschutz. Es gibt auch spezifische Informationen über die verschiedenen Gesellschaften innerhalb der Hermes Gruppe, wie Hermes Germany, Hermes Fulfilment und Hermes Logistik Österreich.'
│   ),
│   (
│   │   'What information do you have about the Hermes Gruppe?',
│   │   'Die Hermes Gruppe ist ein wichtiger Teil der Hermes-Organisation. Hier sind einige Informationen, die ich gefunden habe:\n\n* Die Hermes Gruppe umfasst verschiedene Unternehmen, darunter Hermes Einrichtungs Service, Hermes Fulfilment, Hermes Logistik Österreich, Hermes Germany GmbH und Hermes NexTec.\n* Die Hermes Gruppe bietet eine Vielzahl von Dienstleistungen an, darunter Logistik, Transport, Fulfilment und E-Commerce-Lösungen.\n* Die Hermes Gruppe hat eine starke Präsenz in Europa und bietet Dienstleistungen in vielen Ländern an, darunter Deutschland, Österreich, das Vereinigte Königreich und andere.\n* Die Hermes Gruppe legt Wert auf Nachhaltigkeit und Umweltschutz und bietet grüne Zustelllösungen und energieeffiziente Standorte an.\n* Die Hermes Gruppe unterstützt auch soziale Projekte und engagiert sich für faire Arbeitsbedingungen und die Einhaltung von Sozialstandards.\n\nIch hoffe, diese Informationen sind hilfreich!'
│   ),
│   (
│   │   'What information do you have about the Hermes Germany GmbH?',
│   │   'Ich habe folgende Informationen über die Hermes Germany GmbH:\n\n* Hermes Germany GmbH ist Teil der Hermes Logistik Gruppe Deutschland.\n* Es gibt die Möglichkeit, PaketShop-Partner zu werden.\n* Es gibt die Möglichkeit, Vertragspartner zu werden.\n* Es gibt ein Formular, das ausgefüllt werden kann.\n\nWenn Sie weitere Informationen benötigen, können Sie auch auf der Website https://www.hermesworld.com/de/ueber-uns/hermes-gruppe/hermes-logistik-gruppe-deutschland/hermes-logistik-gruppe-deutschland/ nachlesen.'
│   ),
│   (
│   │   'What information do you have about Hermes Fulfilment?',
│   │   'Es tut mir leid, aber ich habe keine Informationen über Hermes Fulfilment in dem bereitgestellten Kontext gefunden.'
│   )
]

### Long context query

In [36]:
lc_res = lc_query_engine(query)
ic(lc_res)

ic| lc_res: ('Ich habe Daten über die Hermes Gruppe, einem Logistikunternehmen, das '
             'verschiedene Dienstleistungen wie Transport, Logistik, Supply Chain '
             'Solutions und E-Commerce anbietet. Die Daten umfassen Informationen über die '
             'Unternehmenstätigkeit, die verschiedenen Gesellschaften innerhalb der '
             'Gruppe, Karrieremöglichkeiten, News und Pressemitteilungen sowie '
             'Kontaktinformationen.
            '
             '
            '
             'Einige spezifische Daten, die ich habe, sind:
            '
             '
            '
             '* Informationen über die verschiedenen Gesellschaften innerhalb der Hermes '
             'Gruppe, wie z.B. Hermes Germany GmbH, Hermes Logistik Österreich, Hermes UK '
             'und Hermes BorderGuru
            '
             '* Karrieremöglichkeiten bei Hermes, einschließlich Jobangeboten, '
             'Ausbildungsmöglichkeiten und Informationen über die Unterneh

'Ich habe Daten über die Hermes Gruppe, einem Logistikunternehmen, das verschiedene Dienstleistungen wie Transport, Logistik, Supply Chain Solutions und E-Commerce anbietet. Die Daten umfassen Informationen über die Unternehmenstätigkeit, die verschiedenen Gesellschaften innerhalb der Gruppe, Karrieremöglichkeiten, News und Pressemitteilungen sowie Kontaktinformationen.\n\nEinige spezifische Daten, die ich habe, sind:\n\n* Informationen über die verschiedenen Gesellschaften innerhalb der Hermes Gruppe, wie z.B. Hermes Germany GmbH, Hermes Logistik Österreich, Hermes UK und Hermes BorderGuru\n* Karrieremöglichkeiten bei Hermes, einschließlich Jobangeboten, Ausbildungsmöglichkeiten und Informationen über die Unternehmenskultur\n* News und Pressemitteilungen über die Hermes Gruppe und ihre Aktivitäten\n* Kontaktinformationen für Geschäftskunden, Presse und Privatkunden\n* Informationen über die verschiedenen Dienstleistungen, die die Hermes Gruppe anbietet, wie z.B. Sendungsverfolgung, Pa

# Replication query router

The routing is ochstrated by the complexity of the query and the context. Because of the complexity of the query, the RAG will be failed for some reason. The routing will be used to decompose the query into multiple steps. The long-context query will be used for unanswerable questions.

## RAG can be failed by 4 reasons

Accoriding to the paper, there are four reasons why RAG can be failed.

The four reasons include: 

(A) The query requires multi-step reasoning so the results of previous steps are needed to retrieve information for later steps, e.g. ‘‘What nationality is the performer of song XXX’’. 

(B) The query is general, e.g. ‘‘What does the group think about XXX’’, which is challenging for the retriever to formulate a good query. 

(C) The query is long and complex, which is challenging for the retriever to understand. However, answering this kind of questions is arguably, an advantage of LLMs. 

(D) The query is implicit, demanding a thorough understanding of the entire context. For instance, in a lengthy conversational narrative about a space voyage, a question like ‘‘What caused the shadow behind the spaceship?’’ requires readers to connect the dots and deduce the answer, as there is no explicit mention of the shadow when the cause is revealed.

##### Short to say:

(A) The query requires multi-step reasoning.

(B) The query is general. 

(C) The query is long and complex, requiring LLMs. 

(D) The query is implicit, demanding a thorough understanding of the entire context.



## Setup route query engine

### Tools

In [37]:
from llama_index.core.tools import QueryEngineTool

vector_query_engine_tool = QueryEngineTool.from_defaults(
    query_engine=vector_query_engine,
    description="Useful for simple and answerable questions, the data source is German, please think in German context if possible.",
)

multi_steps_query_engine_tool = QueryEngineTool.from_defaults(
    query_engine=multi_steps_query_engine,
    description="Useful for answerable questions that require multiple step reasoning, the data source is German, please think in German context if possible.",
)

lc_query_engine_tool = QueryEngineTool.from_defaults(
    query_engine=lc_query_engine,
    description="Useful for the unanswerable, long and complex or implicit questions that require full context to obtain results, the data source is German, please think in German context if possible.",
)

query_engine_tools = [
    vector_query_engine_tool,
    multi_steps_query_engine_tool,
    lc_query_engine_tool,
]

### Router query engine

In [38]:
from llama_index.core.query_engine import RouterQueryEngine
from llama_index.core.selectors.llm_selectors import LLMSingleSelector
from llama_index.core.response_synthesizers import TreeSummarize

SINGLE_SELECT_PROMPT_TMPL = (
    "Some choices are given below. It is provided in a numbered list "
    "(1 to {num_choices}), "
    "where each item in the list corresponds to a summary.\n"
    "---------------------\n"
    "{context_list}"
    "\n---------------------\n"
    "Using only the choices above and not prior knowledge, return "
    "the choice that is most relevant to the question: '{query_str}'\n"
    "notice: the content is in German and the query might be in English, please translate the query into German and think in German if necessary."
)

router_query_engine = RouterQueryEngine(
    selector=LLMSingleSelector.from_defaults(
        prompt_template_str=SINGLE_SELECT_PROMPT_TMPL
    ),
    query_engine_tools=query_engine_tools,
    summarizer=TreeSummarize(
        streaming=streaming,
        use_async=True,
        verbose=verbose,
    ),
    verbose=verbose,
)

#### Simple question

In [75]:
simply_query = "Tell me the jobs you know at Hermes Group, only the jobs with title and description?"
route_res = router_query_engine.query(simply_query)
print(route_res.response)

Selecting query engine 0: The question is simple and answerable, it only requires a list of job titles and descriptions at Hermes Group, which can be obtained from a specific data source..
Bei Hermes Group gibt es folgende Jobs:

1. **(Junior) Manager (m/w/d) HR Controlling**
   - Beschreibung: Du erstellst Auswertungen, Statistiken und Präsentationen für interne Stakeholder wie HR Kolleg*innen oder Fachbereiche sowie für externe Stakeholder wie Behörden. Du begleitest tatkräftig Prozesse (z.B. Gehaltsrunde) sowie HR-Projekte, um datenbasierte Entscheidungen treffen zu können.

2. **(Senior-) Product Manager (m/w/d)**
   - Beschreibung: Du verantwortest strategisch und operativ einen Teil unseres Produktportfolios (z.B. Retoure) über die gesamte Customer Journey, von der Beauftragung durch den Versender bis zur Zustellung an den Empfänger.


In [76]:
lc_res = lc_query_engine.query(simply_query)
print(route_res.response)

Bei Hermes Group gibt es folgende Jobs:

1. **(Junior) Manager (m/w/d) HR Controlling**
   - Beschreibung: Du erstellst Auswertungen, Statistiken und Präsentationen für interne Stakeholder wie HR Kolleg*innen oder Fachbereiche sowie für externe Stakeholder wie Behörden. Du begleitest tatkräftig Prozesse (z.B. Gehaltsrunde) sowie HR-Projekte, um datenbasierte Entscheidungen treffen zu können.

2. **(Senior-) Product Manager (m/w/d)**
   - Beschreibung: Du verantwortest strategisch und operativ einen Teil unseres Produktportfolios (z.B. Retoure) über die gesamte Customer Journey, von der Beauftragung durch den Versender bis zur Zustellung an den Empfänger.


#### Mult-step requried question

In [77]:
normal_query = "What is the job (Junior) Manager (m/w/d) HR Controlling and (Senior-) Product Manager (m/w/d) TODO, tasks and so on respectively."
route_res = router_query_engine.query(normal_query)
print(route_res.response)

Selecting query engine 1: The question requires multiple step reasoning to understand the job roles and their respective tasks, and the data source is in German, so thinking in German context is necessary..
> Current query: What is the job (Junior) Manager (m/w/d) HR Controlling and (Senior-) Product Manager (m/w/d) TODO, tasks and so on respectively.
> New query: What are the job descriptions for (Junior) Manager (m/w/d) HR Controlling and (Senior-) Product Manager (m/w/d)?
> Current query: What is the job (Junior) Manager (m/w/d) HR Controlling and (Senior-) Product Manager (m/w/d) TODO, tasks and so on respectively.
> New query: What are the main tasks and responsibilities of a (Junior) Manager (m/w/d) HR Controlling and a (Senior-) Product Manager (m/w/d) respectively?
> Current query: What is the job (Junior) Manager (m/w/d) HR Controlling and (Senior-) Product Manager (m/w/d) TODO, tasks and so on respectively.
> New query: What are the specific tasks and responsibilities of a (J

In [78]:
lc_res = lc_query_engine.query(normal_query)
print(route_res.response)

Die Stellenbeschreibungen für (Junior) Manager (m/w/d) HR Controlling und (Senior-) Product Manager (m/w/d) sind wie folgt:

**Junior) Manager (m/w/d) HR Controlling:**

* Du erstellst Auswertungen, Statistiken und Präsentationen für interne Stakeholder wie HR Kolleg*innen oder Fachbereiche sowie für externe Stakeholder wie Behörden
* Du begleitest tatkräftig Prozesse (z.B. Gehaltsrunde) sowie HR-Projekte, um datenbasierte Entscheidungen treffen zu können
* Du arbeitest aktiv an dem Aufbau einer Qualitätskontrolle zwischen unseren Systemen mit
* Neue Ideen und Verbesserungsvorschläge bringst du mit Überzeugungskraft ein

**Senior-) Product Manager (m/w/d):**

* Du verantwortest strategisch und operativ einen Teil unseres Produktportfolios (z.B. Retoure) über die gesamte Customer Journey, von der Beauftragung durch den Versender bis zur Zustellung an den Empfänger
* Du beobachtest und analysierst die Zielgruppen, den Markt und den Wettbewerb, um darauf basierend Produkt-, Preis- und Kom

#### Long and complex question

In [88]:
long_complex_query="Tell me the information of the jobs within your knowledge base, including TODO and the benifits for working in the Hermes Group."
route_res = router_query_engine.query(long_complex_query)
Markdown(route_res.response)

Selecting query engine 2: The question is asking for a broad range of information about jobs within the Hermes Group, including TODO and benefits, which suggests that it requires full context to obtain results. Additionally, the question is somewhat open-ended and implicit, as it doesn't specify what kind of information is being sought. Therefore, choice 3 is the most relevant..


Wir haben verschiedene Jobangebote in unserer Datenbank. Hier sind einige Beispiele:

1. (Junior) Manager (m/w/d) HR Controlling:
	* Unternehmen: Hermes Germany GmbH
	* Standort: Hamburg
	* Berufserfahrung: Junior Level
	* Aufgaben: Erstellung von Auswertungen, Statistiken und Präsentationen für interne und externe Stakeholder, Begleitung von Prozessen und HR-Projekten, Aufbau einer Qualitätskontrolle zwischen Systemen
	* Anforderungen: Studium in Wirtschafts-, Sozialwissenschaften oder Personalmanagement, erste Berufserfahrung, Spaß am Umgang mit Daten und großen Zahlenmengen
	* Benefits: Einsatz führender Technologien und agiler Prozesse, flexible Arbeitszeiten, umfangreiches Weiterbildungsangebot, 15% Rabatt auf otto.de und anderen Shops
2. (Senior-) Product Manager (m/w/d):
	* Unternehmen: Hermes Germany GmbH
	* Standort: Hamburg
	* Berufserfahrung: Mehrjährige Berufserfahrung im Produkt- oder Brand Management
	* Aufgaben: Verantwortung für ein Teil des Produktportfolios, Analyse von Zielgruppen, Markt und Wettbewerb, Initiierung von Produktideen und -launches
	* Anforderungen: Studium mit Schwerpunkt Produktmanagement, mehrjährige Berufserfahrung im Produkt- oder Brand Management, hohe analytische Kompetenz, ausgeprägte Kundenorientierung
	* Benefits: Einsatz führender Technologien und agiler Prozesse, flexible Arbeitszeiten, umfangreiches Weiterbildungsangebot, 15% Rabatt auf otto.de und anderen Shops

Die Hermes Gruppe bietet vielseitige Arbeitsplätze mit langfristigen Perspektiven und täglich wechselnden Herausforderungen in einer sicheren und zukunftsorientierten Branche. Unsere Wertschätzung gilt allen Menschen gleichermaßen - unabhängig von Geschlecht, Nationalität, ethnischer Herkunft, Religion, Behinderung oder sexueller Orientierung.

Wir bieten eine Vielzahl von Benefits, darunter:

* Einsatz führender Technologien und agiler Prozesse
* Flexible Arbeitszeiten, nach Absprache ist auch mobiles Arbeiten möglich
* Umfangreiches Weiterbildungsangebot
* 15% Rabatt auf otto.de und anderen Shops
* Bezuschussung des Deutschlandtickets, Altersvorsorge, JobRad, verschiedene Sport- und Freizeitangebote uvm.
* Vergünstigte und vielfältige Speiseauswahl in unserer modernen Kantine

Wir hoffen, diese Informationen helfen dir bei deiner Jobsuche!